In [17]:
import requests
import bs4

class PasswordError(Exception):
    """Custom exception class for password-related errors during login."""

    def __init__(self, message):
        """
        Initialize a PasswordError instance with a custom error message.

        Args:
            message (str): A descriptive error message.
        """
        super().__init__(message)


class ZJUSession(requests.Session):
    """This class offers ZJU authentication system login ability to requests.Session."""
    def __init__(self, username, password) -> None:
        """Initialize a ZJUSession object for logging in to Zhejiang University's authentication system (zjuam).

        Args:
            username (str): Your ZJU username.
            password (str): Your ZJU password.
        
        Raises:
            PasswordError: If a incorrect password is provided.
            requests.exceptions.RequestException: If a network error occurs during the login process.
            KeyError: If the expected HTML elements or JSON fields are not found.
            ValueError: If there are issues with JSON decoding.

        """
        super().__init__()
        
        self.username = username
        self.password = password
        
        self.login()
        
    def login(self):
        """Log in to Zhejiang University's authentication system (zjuam).

        Raises:
            PasswordError: If a incorrect password is provided.
            requests.exceptions.RequestException: If a network error occurs during the login process.
            KeyError: If the expected HTML elements or JSON fields are not found.
            ValueError: If there are issues with JSON decoding.
            
        """
        # Step 1: GET the login page to obtain cookies, _event_id, and execution values
        login_page_response = self.get('https://zjuam.zju.edu.cn/cas/login')
        login_page_response.raise_for_status()
        soup = bs4.BeautifulSoup(login_page_response.content, features="html.parser")
        event_id = soup.find('input', attrs={'type':'hidden','name':'_eventId'})['value']
        execution = soup.find('input', attrs={'type':'hidden','name':'execution'})['value']
        
        # Step 2: GET the public key needed to encrypt the password
        pub_key_response = self.get('https://zjuam.zju.edu.cn/cas/v2/getPubKey')
        pub_key_response.raise_for_status()
        pub_key = pub_key_response.json()
        
        # Step 3: Encrypt the password using the JavaScript library
        encrypted_password = requests.post('http://localhost:48218/api/encrypt',
                                        json={"exponent":pub_key['exponent'],
                                                "modulus":pub_key['modulus'],
                                                "password":self.password[::-1]
                                                }
                                        ).json()['crypt']

        # Step 4: POST the login request with the encrypted password
        login_response = self.post('https://zjuam.zju.edu.cn/cas/login',data={
                'username': self.username,
                'password': encrypted_password,
                'authcode': '',
                'execution' :execution,
                '_eventId': event_id
            }, allow_redirects=False)
        login_response.raise_for_status()
        if "密码错误" in login_response.content.decode():
            raise PasswordError('Password not correct!')

In [21]:
session = ZJUSession('3230102841','password')


PasswordError: Password not correct!

In [20]:
response = session.get('http://appservice.zju.edu.cn')
print(response.content.decode())

<!DOCTYPE html><html lang=en><head><meta charset=utf-8><meta http-equiv=X-UA-Compatible content="IE=edge"><meta name=viewport content="width=device-width,initial-scale=1,user-scalable=0"><link rel=icon href=http://www.news.zju.edu.cn/_upload/tpl/04/11/1041/template1041/favicon.ico><link rel=stylesheet type=text/css href=https://at.alicdn.com/t/font_1208220_mdy9vikdf6a.css><title>浙江大学</title><style type=text/css>html,
			body {
				width: 100%;
				height: 100%;
			}</style><link href=/assets/css/sy.67492e97.css rel=prefetch><link href=/assets/js/sy.9f5aa611.js rel=prefetch><link href=/assets/css/chunk-common.eff096a0.css rel=preload as=style><link href=/assets/css/chunk-vendors.4da2ccc0.css rel=preload as=style><link href=/assets/css/index.61ea9d96.css rel=preload as=style><link href=/assets/js/chunk-common.7e81bb19.js rel=preload as=script><link href=/assets/js/chunk-vendors.26dbb5d6.js rel=preload as=script><link href=/assets/js/index.e947f5fa.js rel=preload as=script><link href=/ass

In [14]:
response = session.get('http://zdbk.zju.edu.cn/jwglxt/xtgl/login_ssologin.html')
print(response.content.decode())



<!doctype html>
<html lang="zh-CN">
<head>
	<title>教学管理信息服务平台</title>
	





<meta http-equiv="X-UA-Compatible" content="IE=edge" />
<meta name="viewport" content="width=device-width, initial-scale=1">
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta name="Copyright" content="zfsoft" />	
<link rel="icon" href="/jwglxt/logo/favicon.ico" type="image/x-icon" />
<link rel="shortcut icon" href="/jwglxt/logo/favicon.ico" type="image/x-icon" />
<style type="text/css">	
	.active{font-weight: bolder;}
	#navbar-tabs li{ margin-top: 2px;}
	#navbar-tabs li a{ border-top: 2px solid transparent;}
	#navbar-tabs li.active a{border-top: 2px solid #0770cd;}
</style>


	
<!--jQuery核心框架库 -->
<script type="text/javascript" src="/zftal-ui-v5-1.0.2//assets/js/other_jquery/jquery-1.12.4.min.js?ver=28264637"></script>
<script type="text/javascript" src="/zftal-ui-v5-1.0.2//assets/js/jquery-migrate-1.4.1.min.js?ver=28264637"></script>
<!--jQuery浏览器检测 -->
<script type="text/javascrip

In [9]:
response = session.get('https://courses.zju.edu.cn/user/index')
print(response.content.decode())
for i in response.history:
    print(i.url)

<!DOCTYPE html>  <script type="text/javascript" defer="defer" src="/public/downloads/frms-fingerprint.js?custID=124&version=4.3.6&cookieDomain=current&serviceUrl=/public/generate/jsonp"></script>  <script type="application/javascript" src="/js/downloads/extra.js?includes=rd4,ajaxInject&version=1&cookieDomain=current"></script>  <script type="text/javascript">XMLHttpRequest.prototype = Object.getPrototypeOf(new XMLHttpRequest);</script><!doctype html>
<html lang="en-US" version="1.64.46295-release-0b5fa902" host="https://courses.zju.edu.cn:443/" delivery-org="ZJU">
<head>
<script>
    var sentryClientKey = "None";
    sentryClientKey = /^http/.test(sentryClientKey) ? sentryClientKey : '';

    var CONFIGS = {
       "assetsPath": "",
       "sentryClientKey": sentryClientKey,
       "apm": {"DEBUG": false, "ENABLE_APM": true, "ENVIRONMENT": "zju-aliyun \u6d59\u6c5f\u5927\u5b66", "SECRET_TOKEN": "qqEAYoVJ7ocKrAsrJH8rKwc7", "SERVER_URL": "https://apm.tronclass.com.cn", "SERVICE_NAME": "lm